<a href="https://colab.research.google.com/github/tuanbc88/iqa_research/blob/main/RefactorOfIntentClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pyvi

In [ ]:
# -*- coding: utf-8 -*-
 
 
import pandas as pd
import string
import numpy as np
import re, os, string
from pyvi import ViTokenizer
from pyvi.ViTokenizer import tokenize
import tensorflow as tf
from gensim.models.fasttext import FastText 
import json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import *
import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

In [ ]:
def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    return text

def sentence_segment(text):
    sents = re.split("([.?!])?[\n]+|[.?!] ", text)
    return sents

def word_segment(sent):
    sent = tokenize(sent)
    return sent

def normalize_text(text):
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()


def remove_numbers(text_in):
  for ele in text_in.split(): 
    if ele.isdigit():
        text_in = text_in.replace(ele, "@")
  for character in text_in:
    if character.isdigit():
        text_in = text_in.replace(character, "@")
  return text_in


def remove_special_characters(text):
  chars = re.escape(string.punctuation)
  return re.sub(r'['+chars+']', '', text)

 
def preprocess(text_in):  
    text = clean_text(text_in)
    text = remove_special_characters(text)
    text = remove_numbers(text) 
    return text

In [ ]:
filename = '/content/drive/MyDrive/NUCE/NLP/QA/stopwords.csv'
data = pd.read_csv(filename, sep="\t", encoding='utf-8')
list_stopwords = data['stopwords']
def remove_stopword(text):
    pre_text = []
    words = text.split()
    for word in words:
        if word not in list_stopwords:
            pre_text.append(word)
        text2 = ' '.join(pre_text)
    return text2

In [ ]:
max_length_inp = 20
def sentence_embedding(sent):
    content = clean_text(sent) 
    content = remove_special_characters(content) 
    content = word_segment(content)
    content = remove_stopword(normalize_text(content))
    content = remove_numbers(content) 
    inputs = []
    for word in content.split():
      if word in fast_text_model.wv.vocab:
        inputs.append(fast_text_model.wv.get_vector(word))
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                                maxlen=max_length_inp,
                                                                dtype='float32',
                                                                padding='post')
    return inputs



In [ ]:
import json 
    
wiki_data_path = '/content/drive/MyDrive/NUCE/NLP/QA/xaa'
qa_data_path = '/content/drive/MyDrive/NUCE/NLP/QA/intent_db_v2.json'

def read_data(path):
    traindata = []
    sents = open(wiki_data_path, 'r').readlines()
    for sent in sents:
        traindata.append(sent.split())
    with open(qa_data_path) as json_file:
        qa_data = json.load(json_file)  
        for question in qa_data: 
            if 'content' in question :
              content = clean_text(question['content']) 
              content = remove_special_characters(content) 
              content = word_segment(content)
              content = remove_stopword(normalize_text(content))
              content = remove_numbers(content) 
              print(content)
              traindata.append(content.split())
    print("Corpus loaded")
    return traindata, qa_data
train_data, qa_data = read_data(wiki_data_path)


cho em hỏi em có_thể chuyển_đổi kết_quả @ môn_học cùng tên nhưng khác mã được không ạ
em mới làm đơn phúc_khảo gần đây ạ mà @ @ ngày rồi thấy mãi điểm chưa thay_đổi em có_thể xin phép đc xem bài thi của mình đc k ạ nếu em thực_sự sai em chỉ muốn xem_lại bài thi của mình
cho em hỏi là làm_sao để biết kết_quả điểm phúc_khảo môn_học sau khi đã làm đơn phúc_khảo ạ
thưa thầy muốn phúc_khảo điểm môn_học thì làm thế_nào ạ
thầy_cô cho em hỏi là còn thời_gian phúc_tra điểm của kì học vừa_rồi không ạ vì dịch_bệnh không lên trường được nên em đã không phúc_tra được bài ạ
cho em hỏi là muốn phúc_tra điểm thì làm thế_nào ạ
cho em hỏi là thời_hạn phúc_khảo môn_học là bao_giờ ạ em cần làm đơn hay liên_hệ với ai để tiến_hành phúc_tra được ạ
mong thầy cô xem giúp e điểm trên trang đao tạo với ạ hiện_tại điểm đồ_án tốt_nghiệp của e đã được cập nhật lên nhưng có_lẽ do lỗi mà chưa được cộng tổng_điểm trung_bình_tích_lũy và số tín_chỉ của đồ_án tốt_nghiệp
em có thắc_mắc muốn hỏi là @ tín đồ_án tốt_nghiệp đ

In [ ]:
model_fasttext = FastText(size=120, window=10, min_count=2, workers=4, sg=1)


In [ ]:
model_fasttext.build_vocab(train_data)
model_fasttext.train(train_data, total_examples=model_fasttext.corpus_count, epochs=model_fasttext.iter)
model_fasttext.wv.save("/content/drive/MyDrive/NUCE/NLP/QA/model/fasttext_gensim.model")

KeyboardInterrupt: ignored

In [ ]:
from gensim.models import KeyedVectors

fast_text_model = KeyedVectors.load('/content/drive/MyDrive/NUCE/NLP/QA/model/fasttext_gensim.model')

In [ ]:
import json 

def read_data(qa_data_input):
  sentences = [] 
  labels = []
  intents = [] 
  for question in qa_data: 
    if 'intent' in question:
      if question['intent'] not in intents: 
        intents.append(question['intent']) 
      sentences.append(sentence_embedding(question['content'])[0])
      labels.append(intents.index(question['intent']))
  print("Corpus loaded")
  return sentences, labels, intents

 
sentences, labels, intents = read_data(qa_data)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # This is added back by InteractiveShellApp.init_path()


Corpus loaded


In [ ]:
!nvidia-smi

In [ ]:
input_text = fast_text_model.wv.get_vector("hôm_nay") 
print(input_text.shape)

In [ ]:
import numpy as np
import json
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import *

In [ ]:
#model A
model = Sequential()

model.add(LSTM(128, input_shape=(max_length_inp, fast_text_model.vector_size)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(intents), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# #Model B
# model = Sequential()
# model.add(Bidirectional(GRU(128, input_shape=(max_length_inp,fast_text_model.vector_size))))
# model.add(Dense(64, activation = "tanh"))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation = "tanh"))
# model.add(Dropout(0.5))
# model.add(BatchNormalization())
# model.add(Dense(len(intents), activation='softmax'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
def split_balanced(data, target, test_size=0.2):

    classes = np.unique(target)
    # can give test_size as fraction of input data size of number of samples
    if test_size<1:
        n_test = np.round(len(target)*test_size)
    else:
        n_test = test_size
    n_train = max(0,len(target)-n_test)
    n_train_per_class = max(1,int(np.floor(n_train/len(classes))))
    n_test_per_class = max(1,int(np.floor(n_test/len(classes))))

    ixs = []
    for cl in classes:
        if (n_train_per_class+n_test_per_class) > np.sum(target==cl):
            # if data has too few samples for this class, do upsampling
            # split the data to training and testing before sampling so data points won't be
            #  shared among training and test data
            splitix = int(np.ceil(n_train_per_class/(n_train_per_class+n_test_per_class)*np.sum(target==cl)))
            ixs.append(np.r_[np.random.choice(np.nonzero(target==cl)[0][:splitix], n_train_per_class),
                np.random.choice(np.nonzero(target==cl)[0][splitix:], n_test_per_class)])
        else:
            ixs.append(np.random.choice(np.nonzero(target==cl)[0], n_train_per_class+n_test_per_class,
                replace=False))

    # take same num of samples from all classes
    ix_train = np.concatenate([x[:n_train_per_class] for x in ixs])
    ix_test = np.concatenate([x[n_train_per_class:(n_train_per_class+n_test_per_class)] for x in ixs])

    X_train = data[ix_train,:]
    X_test = data[ix_test,:]
    y_train = target[ix_train]
    y_test = target[ix_test]

    return X_train, X_test, y_train, y_test

In [ ]:
trainX = np.array(sentences)

trainy = to_categorical(np.array(labels), len(intents))

X_train, X_test, y_train, y_test = split_balanced(trainX, trainy, test_size=0.1)


In [ ]:
import datetime
logdir = os.path.join("logs_dev_3", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


In [ ]:
tf.test.is_gpu_available()

In [ ]:
model.fit(X_train, y_train, epochs=300, batch_size=10,verbose=1,  validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

Epoch 1/300
230/230 [==============================] - 6s 20ms/step - loss: 2.1155 - accuracy: 0.2018 - val_loss: 1.8002 - val_accuracy: 0.3362
Epoch 2/300
230/230 [==============================] - 4s 18ms/step - loss: 1.7810 - accuracy: 0.3503 - val_loss: 1.5388 - val_accuracy: 0.4913
Epoch 3/300
230/230 [==============================] - 4s 18ms/step - loss: 1.4580 - accuracy: 0.5156 - val_loss: 1.4652 - val_accuracy: 0.5261
Epoch 4/300
230/230 [==============================] - 4s 18ms/step - loss: 1.2682 - accuracy: 0.5744 - val_loss: 1.1977 - val_accuracy: 0.6202
Epoch 5/300
230/230 [==============================] - 4s 17ms/step - loss: 1.1051 - accuracy: 0.6458 - val_loss: 1.1313 - val_accuracy: 0.6446
Epoch 6/300
230/230 [==============================] - 4s 18ms/step - loss: 0.9449 - accuracy: 0.6930 - val_loss: 1.0536 - val_accuracy: 0.6498
Epoch 7/300
230/230 [==============================] - 4s 18ms/step - loss: 0.8444 - accuracy: 0.7271 - val_loss: 1.0451 - val_accuracy:

KeyboardInterrupt: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs_dev_3

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=10)
print("test loss, test acc:", results)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix




In [ ]:
yhat_classes = model.predict_classes(X_test, verbose=0)
y_test_true = np.array([np.where(label ==1)[0][0] for label in y_test])
f1 = f1_score(yhat_classes, y_test_true, average='weighted')
print(f1)

In [ ]:
label_list = np.array(labels)
count_classes = [np.count_nonzero(label_list == i) for i in range(len(intents)) ]
print(count_classes)


In [ ]:


objects = (intent for intent in intents)
y_pos = np.arange(len(intents))
performance = count_classes

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Quantity')
plt.title('Number of intents')

plt.show()